In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from implementations import *
import seaborn as sns
import pandas as pd
import pickle

## Data Cleaning

#### Check missing column values

In [ ]:
# Utility functions
def load_data():
    """load data."""
    f = open(f"dataset/x_train.csv")
    features = f.readline()
    feature_names = features.split(',')
    data = np.loadtxt(f"dataset/x_train.csv", delimiter=",", skiprows=1, dtype=str)
    return data,feature_names

def convert_row_to_float(row):
    """Convert values in row to float or np.nan."""
    new_row = []
    for item in row:
        try:
            new_row.append(float(item))
        except ValueError:
            new_row.append(np.nan)
    return np.array(new_row)

def convert_all_rows(data):
    """Convert all rows to float or np.nan."""
    new_data = []
    for row in data:
        new_data.append(convert_row_to_float(row))
    return np.array(new_data)

def column_NAN(array):
    nan=0
    for i in range(len(array)):
        if np.isnan(array[i]):
                nan += 1
    return nan

In [ ]:
def train_validation_split(data, ratio, seed):
    """Split data into training and validation set."""
    np.random.seed(seed)
    np.random.shuffle(data)
    split_index = int(len(data) * ratio)
    return data[:split_index], data[split_index:]

def k_fold_split(data, k, seed):
    """Split data into k folds."""
    np.random.seed(seed)
    np.random.shuffle(data)
    return np.array_split(data, k)

def standardize_data(data):
    """Standardize data."""
    mean = np.nanmean(data, axis=0)
    std = np.nanstd(data, axis=0)
    return (data - mean) / std

In [ ]:
numpy_data, features = load_data()
import pandas as pd
pandas_df = pd.read_csv("dataset/x_train.csv")

In [ ]:
numpy_data, features = load_data()

In [ ]:
display(pandas_df)
print(numpy_data)

In [ ]:
first_row = numpy_data[0]
new_first_row = convert_row_to_float(first_row)
# Compare the values of the firs row in the numpy array and the pandas dataframe
for i, j in zip(pandas_df.iloc[0,:].values, new_first_row):
    print(f"Pandas value: {i} | Numpy value: {j}")

In [ ]:
data = convert_all_rows(numpy_data)
data

In [ ]:
data[:,0]

In [ ]:
# Count the number of NaNs in each column
number_of_nans = pandas_df.isnull().sum()
# Sort in ascending order
number_of_nans.sort_values(inplace=True, ascending=False)
for index, value in zip(number_of_nans.index, number_of_nans.values):
    print(f"Column {index} has {value} NaNs")
# This shows that for some columns we essentially have no meaningful data.

In [ ]:
for i in features:
    print(f"feature {i} has {column_NAN(data[:,features.index(i)])} NaNs")

In [ ]:
column_NAN(data[:,features.index('COLGHOUS')])

In [ ]:
print(f"Total number of observations: {len(pandas_df)}")
print(f" 10% of the number of observations: {round(len(pandas_df) * 0.1)}")

Here we can decide which columns to drop based on the number of nans in the <span style="color:red"> test set </span> . \
Since at the end of the day we want to make predictions on the test set.

In [ ]:
# Keep only the columns with less than 10% NaNs
columns_to_keep = []
for index, value in zip(number_of_nans.index, number_of_nans.values):
    if value < round(len(pandas_df) * 0.1):
        columns_to_keep.append(index)
# Keep only the columns with less than 10% NaNs
data_with_few_nans = pandas_df.loc[:, columns_to_keep]

# Save the columns_to_keep list to a pickle file
with open("columns_to_keep_at_90.pkl", "wb") as f:
    pickle.dump(columns_to_keep, f)

In [ ]:
print(pandas_df.shape)
print(data_with_few_nans.shape)

In [ ]:
# Cound the number of unique values in each column
number_of_unique_values = data_with_few_nans.nunique()
# Sort in ascending order
number_of_unique_values.sort_values(inplace=True, ascending=False)
for index, value in zip(number_of_unique_values.index, number_of_unique_values.values):
    print(f"Column {index} has {value} unique values")

<span style="color:red"> Here we also need to make a decision what threshold to use for columns to drop. </span> 

Could be that we use:
- Even higher threshold than 90%
- A smaller threshold and try to interpolate the data from other features.

Optimally: I think what we want is to <span style="color:red"> have train -test sets independent of what columns we dropped </span> that way we could compare the loss across models which use different numbers of features.

## a) Carry on Datanalysis only with columns having > 90% of values

In [ ]:
# Count the number of missing values for each row
number_of_nans_per_row = data_with_few_nans.isnull().sum(axis=1)
# Sort in ascending order
number_of_nans_per_row.sort_values(inplace=True, ascending=False)
for index, value in zip(number_of_nans_per_row.index, number_of_nans_per_row.values):
    print(f"Row {index} has {value} NaNs")

# Plot the histogram of the number of missing values per row
plt.figure(figsize=(10, 5))
plt.hist(number_of_nans_per_row.values, bins=20)
plt.xlabel("Number of NaNs")
plt.ylabel("Count")
plt.title("Number of NaNs per row")

We see that for most rows we actually have around 0 Missing values, which is pretty good for the Test set.

We have a number of choices here: 
- Drop rows which have say more than 10 missing value.
- Replace all the Nan values with zeros.
- Replace all the Nan values with medians, means, modes
- Use some sort of interpolation technique

In [ ]:
# Read in the y_train data
y_train = pd.read_csv("dataset/y_train.csv")

In [ ]:
# Now remove the columns with more than 10% of Nans using  the columns_to_keep list
x_train_reduced_90 = data_with_few_nans

# Again count the number of NaNs in each column
number_of_nans = x_train_reduced_90.isnull().sum()
number_of_nans.sort_values(inplace=True, ascending=False)
for index, value in zip(number_of_nans.index, number_of_nans.values):
    print(f"Column {index} has {value} NaNs")

Looks quite similar to the Test data fortunately

In [ ]:
def drop_rows_by_missing_values(data, threshold):
    """Drop rows with more than threshold missing values."""
    number_of_nans_per_row = data.isnull().sum(axis=1)
    rows_to_drop = number_of_nans_per_row[number_of_nans_per_row > threshold].index
    new_data = data.drop(rows_to_drop)
    return new_data

In [ ]:
def replace_missing_values_with_mean(data):
    """Replace missing values with the mean of the column."""
    new_data = data.fillna(data.mean())
    return new_data

def replace_missing_values_with_median(data):
    """Replace missing values with the median of the column."""
    new_data = data.fillna(data.median())
    return new_data

def replace_missing_values_with_mode(data):
    """Replace missing values with the mode of the column."""
    new_data = data.fillna(data.mode().iloc[0])
    return new_data

def replace_missing_values_with_zero(data):
    """Replace missing values with zero."""
    new_data = data.fillna(0)
    return new_data

#### Scenario 1)
#### Do not Drop data, replace the missing values in each row with column means

In [ ]:
# Standardize the data
df_train_reduced_90_mean = standardize_data(x_train_reduced_90)
#Fill in the missing values with the mean of the column
df_train_reduced_90_mean = replace_missing_values_with_mean(df_train_reduced_90_mean)
# Split the data into training and validation set
train_data, validation_data = train_validation_split(df_train_reduced_90_mean.values, 0.8, 42)
print(train_data.shape)
print(validation_data.shape)

Possibly also add a bias term to the features

In [ ]:
# Initialize the weights using the normal distribution
np.random.seed(42)
initial_weights = np.random.normal(size=train_data.shape[1])
# Fit the model using Linear Regression with Gradient Descent
w_ols_sgd, loss = mean_squared_error_gd(y_train.values, train_data, initial_w=initial_weights, max_iters=1000, gamma=0.1)